In [33]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.trend import SMAIndicator

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2011-10-01"
end_date="2021-10-01"
intr="1wk"
txt = "{:%}"
wind=14
for i in range(0,55):
    try:
        stk=sip[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()

        dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
        dfw = yf.download(stk, start_date, end_date,interval='1wk').dropna()
        #dfm = yf.download(stk, start_date, end_date,interval='1mo').dropna()

        #stoch_rsi_m=StochRSIIndicator(close=dfm.Close,window=14,smooth1=3, smooth2=3)


        dfd['efid']=ForceIndexIndicator(close=dfd.Close,volume=dfd.Volume,window=wind,fillna=True).force_index().shift(1)
        dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()

        dfw['efiw']=ForceIndexIndicator(close=dfw.Close,volume=dfw.Volume,window=wind,fillna=True).force_index().shift(1)
        dfw['efi_rsiw']=RSIIndicator(close=dfw.efiw,window=wind,fillna=True).rsi()

        dfd["kd"]=StochRSIIndicator(close=dfd.Close,window=wind,smooth1=3, smooth2=3).stochrsi_k().shift(1)    
        dfw["kw"]=StochRSIIndicator(close=dfw.Close,window=wind,smooth1=3, smooth2=3).stochrsi_k().shift(1)
        #dfm["km"]=stoch_rsi_m.stochrsi_k().shift(1)

        #print(dfw.tail())
        dfd['sma200']=SMAIndicator(close=dfd.Close,window=200,fillna=True).sma_indicator().shift(1)
        dfd['sma50']=SMAIndicator(close=dfd.Close,window=50,fillna=True).sma_indicator().shift(1)
        dff=dfw.join(dfd['kd']).join(dfd['efi_rsid']).join(dfd['sma200']).join(dfd['sma50']).dropna()
        #print(dff.tail())


        dff["Chng"]=(dff.Close-dff.Open)

        #k_thresh=np.percentile(df['k'].dropna(),90)
        #print(k_thresh)
        perd=70
        perv=70

        conditions = [ 
        (dff["kw"]<=np.percentile(dff['kw'],100-perv))
        & (dff['kd']<=np.percentile(dff['kd'],100-perd)) 
        & (dff['efi_rsiw']<=np.percentile(dff['efi_rsiw'],100-perv))
        & (dff['efi_rsid']<=np.percentile(dff['efi_rsid'],100-perd))
        & (dff['Open']>=dff['sma200'])
        & (dff['sma50']>=dff['sma200'])
        ,

        (dff["kw"]>np.percentile(dff['kw'],perv)) 
        & (dff['kd']>np.percentile(dff['kd'],perd))
        & (dff['efi_rsiw']>np.percentile(dff['efi_rsiw'],perv)) 
        & (dff['efi_rsid']>np.percentile(dff['efi_rsid'],perd)) 
        & (dff['Open']<dff['sma200'])
        & (dff['sma50']<dff['sma200'])
        ]

        choices = [
        (dff['Chng'])
        ,
        (-dff['Chng'])
        ]

        dff['trade']=np.select(conditions,choices,default=0)
        base=dff['Close'][len(df.index)-1]-dff['Open'][0]
        strat=np.sum(dff.trade)
        winr=np.count_nonzero(dff.trade>0)
        losr=np.count_nonzero(dff.trade<0)
        leng=len(dff.iloc[:,0])
        winrb=np.count_nonzero(dff.Chng>0)/leng

        temp=pd.Series([stk,base,strat,winrb,(winr/(losr+winr)),((winr+losr)/leng)])
        dfout=dfout.append(temp,ignore_index=True)
        #print(df.tail())
    except:
        pass
dfout.columns=['stk','base_return','strat_return','base_winrate','strat_winrate','strat_traderate']
print(dfout)
print(txt.format(np.average(dfout.base_winrate)))
print(txt.format(np.average(dfout.strat_winrate)))
print(txt.format(np.average(dfout.strat_winrate)-(np.average(dfout.base_winrate))))
print(txt.format(np.average(dfout.strat_traderate)))
#dfout.to_csv('stochrsi_week_edge.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********